In [4]:
### Imports
import pandas as pd
import numpy as np




In [416]:
# Extracting the file
def extract(data):
    df = pd.read_excel(data, index_col=0)
    return df

# Transformations
# Grab column names from dataframe


def get_cols(data):
    column_names = list(data.columns)
    return column_names

# Correct the date format


def right_date(data, col_name):
    import datetime
    if col_name == "visit_start_date":
        data[col_name] = pd.to_datetime(data[col_name], dayfirst=True)
    elif col_name == "visit_end_date":
        data[col_name] = pd.to_datetime(data[col_name])
    elif col_name == "pet_dob":
        data[col_name] = pd.to_datetime(data[col_name], dayfirst=True)
    print(type(data[col_name][1]))
    return data

# Duration of visit


def duration(data):
    durations = []
    for i in range(len(data)):
        try:
            durations.append(
                (data["visit_end_date"][i+1] - data["visit_start_date"][i+1]).days)
        except:
            durations.append(0)
    data['durations'] = durations
    #print(data.loc[data.durations < 0])
    return data

def current_age(data):
    from dateutil.relativedelta import relativedelta
    age = []
    for i in range(len(data)):
            try:
                age.append((pd.to_datetime('today')- data.pet_dob).days / 365)
            except:
                age.append(0)
    data['age'] = age
    return data


# Removing currency symbols
def clean_prices(data):
    for i in range(len(data)):
        if type(data['visit_cost'][i+1]) == str:
            data['visit_cost'][i+1] = data['visit_cost'][i+1][1:]
        else:
            continue

    return None


# Split tables
## individual tables
def split_table(data):
    hospital_table = data[['hospital_id','hospital']].drop_duplicates(subset='hospital_id', keep="first").reset_index(drop = True)
    owner_table = data[['owner_id','pet_id','first_name','last_name','email','address','city','postcode']].drop_duplicates(subset='owner_id', keep="first").reset_index(drop = True)
    pet_table = data[['owner_id', 'pet_id', 'pet_name', 'species', 'breed', 'pet_dob']].drop_duplicates(subset='pet_id', keep="first").reset_index(drop = True)
    visit_table = data[['hospital_id','owner_id', 'procedure_code','visit_start_date','visit_end_date','visit_cost']].reset_index()
    procedure_table = data[['procedure_code','procedure_desc']].drop_duplicates(subset='procedure_code', keep="first").reset_index(drop = True)
    return hospital_table, owner_table, pet_table, visit_table, procedure_table
  
        
#Load
##save each table into individual excel sheets
def write_excel(file_loc, tables,table_names):
    import os.path
    for i in range(len(tables)):
        table =  tables[i]
        table_name = table_names[i]
        missing_table = table[table.isna().any(axis= 1)]
        if os.path.exists(file_loc):
            with pd.ExcelWriter(file_loc, engine="openpyxl",mode='a') as writer: 
                table.to_excel(writer, sheet_name = table_name,  index= False)
                missing_table.to_excel(writer, sheet_name = 'missing_'+ table_name, index= False)
        else:
            table.to_excel(file_loc, sheet_name = table_name, index = False)
            missing_table.to_excel(file_loc, sheet_name = 'missing_'+ table_name, index= False)



In [417]:
data_file = ".\modelling-1(3549).xlsx"
df = extract(data_file)
#print(df.head(2))

clean_prices(df)

hospital_table, owner_table, pet_table, visit_table, procedure_table = split_table(df)
visit_table = right_date(visit_table,"visit_start_date")
visit_table= right_date(visit_table,"visit_end_date")
visit_table = duration(visit_table)
pet_table = right_date(pet_table, 'pet_dob')
pet_table = current_age(pet_table)
print(visit_table.head(5), visit_table.info())

tables =[hospital_table, owner_table, pet_table, visit_table, procedure_table]
table_names =['hospital_table', 'owner_table', 'pet_table','visit_table', 'procedure_table']



file_loc = './results.xlsx'
write_excel(file_loc, tables, table_names)




<ipython-input-416-e3756f14e027>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['visit_cost'][i+1] = data['visit_cost'][i+1][1:]


<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   visit_id          1000 non-null   int64         
 1   hospital_id       1000 non-null   int64         
 2   owner_id          1000 non-null   int64         
 3   procedure_code    1000 non-null   object        
 4   visit_start_date  987 non-null    datetime64[ns]
 5   visit_end_date    987 non-null    datetime64[ns]
 6   visit_cost        1000 non-null   object        
 7   durations         987 non-null    float64       
dtypes: datetime64[ns](2), float64(1), int64(3), object(2)
memory usage: 62.6+ KB
   visit_id  hospital_id  owner_id procedure_code visit_start_date  \
0         1           86       194        02114K9 